In [505]:
import pandas as pd

In [506]:
from pymorphy3 import MorphAnalyzer
from nltk.corpus import stopwords

morph = MorphAnalyzer()
stopwords_ru = stopwords.words("russian")

def morphe(str):
    return ' '.join([morph.normal_forms(w)[0] for w in str.split() if w not in stopwords_ru])

def morphem(df):
    from nltk.corpus import stopwords
    morph = MorphAnalyzer()
    stopwords_ru = stopwords.words("russian")
    df = df.copy()
    df['query'].apply(lambda str:' '.join([morph.normal_forms(w.lower())[0] for w in str.split() if w not in stopwords_ru]))
    return df

In [507]:
fillna_columns = [
    "занятость",
    "по должности-лемме",
    "по дополнительному признаку",
    "по условиям",
    "общие фразы",
]

In [508]:
df = pd.read_csv("answers.csv", index_col='Unnamed: 0')
for col in fillna_columns:
    df.fillna({col: f'{col}_нет'}, inplace=True)

In [509]:
onehot_columns = ['по должности-лемме', 'общие фразы']
multilabel_columns = ['занятость', 'по дополнительному признаку',  'по условиям']
vectorizer_columns = 'query'

In [510]:
def split(x):
    return [i for i in set([x.lower() for x in x.split(',')]) if len(i)>0]

for col in multilabel_columns:
    df[col] = df[col].apply(lambda x: split(x))

In [511]:
X = df[['query']]
y = df.drop(columns=['query'])

In [512]:
y

,занятость,по должности-лемме,по дополнительному признаку,по условиям,общие фразы
0,[занятость_нет],по должности-лемме_нет,[по дополнительному признаку_нет],[по условиям_нет],общая фраза
1,[занятость_нет],по должности-лемме_нет,[по дополнительному признаку_нет],[по условиям_нет],общая фраза
2,[занятость_нет],Рабочий,[по дополнительному признаку_нет],[по условиям_нет],общие фразы_нет
3,[на неполный день],Водитель,[по дополнительному признаку_нет],[по условиям_нет],общие фразы_нет
4,[занятость_нет],Водитель,[по дополнительному признаку_нет],[по условиям_нет],общие фразы_нет
...,...,...,...,...,...
14240,[занятость_нет],по должности-лемме_нет,[по дополнительному признаку_нет],[по условиям_нет],общая фраза
14241,[занятость_нет],по должности-лемме_нет,[по дополнительному признаку_нет],[по условиям_нет],общая фраза
14242,[занятость_нет],по должности-лемме_нет,[по дополнительному признаку_нет],[по условиям_нет],общая фраза
14243,[занятость_нет],по должности-лемме_нет,[по дополнительному признаку_нет],[по условиям_нет],общая фраза


In [513]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.compose import ColumnTransformer

from sklearn.feature_extraction.text import CountVectorizer


In [514]:
from sklearn.pipeline import FunctionTransformer


preprocessor_y = ColumnTransformer(
    transformers=[

        (
            "onehot",
            OneHotEncoder(sparse_output=False, handle_unknown='ignore'),
            onehot_columns,
        ),
        ('multilabel0', CountVectorizer(analyzer=set), multilabel_columns[0]),
        ("multilabel1", CountVectorizer(analyzer=set), multilabel_columns[1]),
        ("multilabel2", CountVectorizer(analyzer=set), multilabel_columns[2]),
        # ("vectorizer", CountVectorizer(), vectorizer_columns),
    ]
)

preprocessor_x = Pipeline([
    ('morphem', FunctionTransformer(morphem)),
    ('vectorizer', ColumnTransformer([('vectorize', CountVectorizer(), vectorizer_columns)])),
])

In [515]:
from sklearn.base import BaseEstimator, TransformerMixin


class InverseTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, pipe):
        self.pipe = pipe

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        arr = X[0]
        inverse_transformers = self.pipe.transformers_

        len1 = len(inverse_transformers[0][1].get_feature_names_out())
        len2 = len(list(inverse_transformers[1][1].vocabulary_.keys()))
        len3 = len(list(inverse_transformers[2][1].vocabulary_.keys()))

        res = inverse_transformers[0][1].inverse_transform([arr[:len1]])
        res2 = inverse_transformers[1][1].inverse_transform([arr[len1 : len1 + len2]])
        res3 = inverse_transformers[2][1].inverse_transform([arr[len1 + len2 : len1 + len2 + len3]])
        res4 = inverse_transformers[3][1].inverse_transform([arr[len1 + len2 + len3:]])

        output = dict(zip(inverse_transformers[0][1].feature_names_in_, res[0]))
        output["занятость"] = res2[0].tolist()
        output["по дополнительному признаку"] = res3[0].tolist()
        output["по условиям"] = res4[0].tolist()

        for key, value in output.items():
            if value is None:
                output[key] = 'Нет'
            if isinstance(value, list):
                for idx, item in enumerate(value):
                    output[key][idx] = item.replace(f'{key}_нет', 'Нет')
            else:
                output[key] = output[key].replace(f'{key}_нет', 'Нет')
        return output

In [516]:
from sklearn.neural_network import MLPClassifier

In [517]:
# pipeline = Pipeline([
#     ("preprocessor", preprocessor),
#     ("model", MLPClassifier(max_iter=400)),
#     ("inverse", FunctionTransformer())
# ])

In [518]:
y_prep = preprocessor_y.fit_transform(y)
y_prep

array([[0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.]])

In [519]:
# x_prep = preprocessor_x.fit_transform(X)
# x_prep.todense()

In [520]:
model = MLPClassifier(max_iter=400)

In [521]:
pipeline = Pipeline(steps=[
    ("preprocessor", preprocessor_x),
    ("model", model),
    # ("inverse", InverseTransformer(preprocessor_y))
])

inverser = InverseTransformer(preprocessor_y)

In [522]:
pipeline.fit(X, y_prep)

Pipeline(steps=[('preprocessor',
                 Pipeline(steps=[('morphem',
                                  FunctionTransformer(func=<function morphem at 0x00000218C829DA80>)),
                                 ('vectorizer',
                                  ColumnTransformer(transformers=[('vectorize',
                                                                   CountVectorizer(),
                                                                   'query')]))])),
                ('model', MLPClassifier(max_iter=400))])

In [523]:
inverser = InverseTransformer(preprocessor_y)

In [538]:
example_qeury = "работа грузчиком с ежедневной оплатой"

frame = pd.DataFrame({'query': [example_qeury]})
preds = pipeline.predict(frame)

In [539]:
# example_prep = preprocessor_x.transform(frame)
# preds = model.predict(example_prep)
# preds = pipeline.predict(frame)
result = inverser.transform(preds)
# cut = preds[0]
result

{'по должности-лемме': 'Грузчик',
 'общие фразы': 'Нет',
 'занятость': ['Нет'],
 'по дополнительному признаку': ['Нет'],
 'по условиям': ['с ежедневной оплатой']}

In [540]:
import dill

with open('model.pkl', 'wb') as file:
    pipe_data = {
        'pipeline': pipeline,
        'inverser': inverser
    }
    dill.dump(pipe_data, file)

In [527]:
# inverse_transformers = preprocessor_y.transformers_

In [528]:
# list(inverse_transformers[1][1].vocabulary_.keys())

In [529]:
# res = inverse_transformers[0][1].inverse_transform([cut[:303]])
# res

In [530]:
# output = dict(zip(inverse_transformers[0][1].feature_names_in_, res[0]))

In [531]:
# res2 = inverse_transformers[1][1].inverse_transform([cut[303:316]])
# output['занятость'] = res2[0].tolist()

In [532]:
# res2

In [533]:
# res3 = inverse_transformers[2][1].inverse_transform([cut[316:]])
# output['по дополнительному признаку'] = res3[0].tolist()

In [534]:
# output